In [1]:
#This code imports the three source files from usafacts
#It produces multiple dataframes:
##raw
#1. cases_by_county_df: confirmed cases by county, by day from 1/22/20 to 6/15/20, including a column for country population.
#2. cases_by_state_df: a group-by of #1, organised into US States.
#3. deaths_by_county_df: confirmed deaths by county, by day from 1/22/20 to 6/15/20, including a column for country population.
#4. deaths_by_state_df: a group-by of #3, organised into US States.
#5. rates_by_county_df: #3 divivded by #1, to get deaths / cases per day by county.
#6. rates_by_state_df: #4 divivded by #2, to get deaths / cases per day by state.
##per capita
#7. cases_by_county_per_capita: #1 divided by 2019 census population
#8. deaths_by_county_per_capita: #3 divided by 2019 census population
#9. rates_by_county_per_capita: #5 divided by 2019 census population
#10. cases_by_state_per_capita: #2 divided by 2019 census population
#11. deaths_by_state_per_capita: #4 divided by 2019 census population
#12. rates_by_state_per_capita: #6 divided by 2019 census population

In [2]:
# Dependencies and Setup
import os
import pandas as pd

In [3]:
#import csv files from output path
file_path = os.path.join('source_data', 'covid_confirmed_usafacts.csv')
cases_by_county_df = pd.read_csv(file_path, encoding="utf-8")

file_path = os.path.join('source_data', 'covid_deaths_usafacts.csv')
deaths_by_county_df = pd.read_csv(file_path,encoding="utf-8")

file_path = os.path.join('source_data', 'covid_county_population_usafacts.csv')
county_pop_df = pd.read_csv(file_path, encoding="utf-8")

In [4]:
#create new columns for county+state (because lots of duplicate county names in different states)
county_pop_df["County&State"] = county_pop_df['County Name'] + ',' + county_pop_df['State']
deaths_by_county_df["County&State"] = deaths_by_county_df['County Name'] + ',' + deaths_by_county_df['State']
cases_by_county_df["County&State"] = cases_by_county_df['County Name'] + ',' + cases_by_county_df['State']

In [5]:
#data clean up - some county names in deaths_by_county_df are not the same in other dataframes.
deaths_by_county_df['County&State'] = deaths_by_county_df['County&State'].replace({'Broomfield County,CO': 'Broomfield County and City,CO', 
                                                                       'Mathews County,VA': 'Matthews County,VA',
                                                                        'Lac qui Parle County,MN': 'Lac Qui Parle County,MN',          
                                                                         'Alexandria city,VA': 'Alexandria City,VA',
                                                                                   'Charlottesville city,VA': 'Charlottesville City,VA',
                                                                                   'Chesapeake city,VA': 'Chesapeake City,VA',
                                                                                   'Danville city,VA': 'Danville City,VA',
                                                                                   'Fredericksburg city,VA': 'Fredericksburg City,VA',
                                                                                   'Harrisonburg city,VA': 'Harrisonburg City,VA',
                                                                                   'Manassas city,VA': 'Manassas City,VA',
                                                                                   'Norfolk city,VA': 'Norfolk City,VA',
                                                                                   'Portsmouth city,VA': 'Portsmouth City,VA',
                                                                                   'Richmond city,VA': 'Richmond City,VA',
                                                                                   'Suffolk city,VA': 'Suffolk City,VA',
                                                                                   'Virginia Beach city,VA': 'Virginia Beach City,VA'
                                                                                  })

In [6]:
#1. cases_by_county_df
#add county population to cases_by_county_df
cases_by_county_df = pd.merge(cases_by_county_df, county_pop_df, on="County&State", how="outer")
#tidy up
cases_by_county_df.rename(columns={'countyFIPS_x':'countyFIPS','County Name_x': 'County Name', "State_x":'State',"population_x":"population"}, inplace=True)
del cases_by_county_df['countyFIPS_y']
del cases_by_county_df['State_y']
del cases_by_county_df['County Name_y']
cases_by_county_df = cases_by_county_df.set_index("County&State")
cases_by_county_df.to_csv('output_data/cases_by_county_df.csv', index=True, header=True)

In [7]:
#3. deaths_by_county_df
#add county population to deaths_by_county_df
deaths_by_county_df = pd.merge(deaths_by_county_df, county_pop_df, on="County&State", how="outer")
#tidy up
deaths_by_county_df.rename(columns={'countyFIPS_x':'countyFIPS','County Name_x': 'County Name', "State_x":'State',"population_x":"population"}, inplace=True)
del deaths_by_county_df['countyFIPS_y']
del deaths_by_county_df['State_y']
del deaths_by_county_df['County Name_y']
deaths_by_county_df = deaths_by_county_df.set_index("County&State")
deaths_by_county_df.to_csv('output_data/deaths_by_county_df.csv', index=True, header=True)

In [8]:
#2. cases_by_state_df
#4. deaths_by_state_df
#groupby state
cases_by_state_df = cases_by_county_df.groupby(['State'])
cases_by_state_df = cases_by_state_df.sum()
cases_by_state_df.to_csv('output_data/cases_by_state_df.csv', index=True, header=True)

deaths_by_state_df = deaths_by_county_df.groupby(['State'])
deaths_by_state_df = deaths_by_state_df.sum()
deaths_by_state_df.to_csv('output_data/deaths_by_state_df.csv', index=True, header=True)


In [9]:
#5. rates_by_county_df
#divide deaths by cases for county

#copy dataframe and delete columns that are strings
deaths_by_county_df_no_string = deaths_by_county_df
del deaths_by_county_df_no_string['countyFIPS']
del deaths_by_county_df_no_string['County Name']
del deaths_by_county_df_no_string['State']
del deaths_by_county_df_no_string['stateFIPS']
#copy dataframe and delete columns that are strings
case_by_county_df_no_string = cases_by_county_df
del case_by_county_df_no_string['countyFIPS']
del case_by_county_df_no_string['County Name']
del case_by_county_df_no_string['State']
del case_by_county_df_no_string['stateFIPS']

rates_by_county_df = deaths_by_county_df_no_string.div(case_by_county_df_no_string)
#rates_by_county_df=rates_by_county_df.fillna(0)
rates_by_county_df.to_csv('output_data/rates_by_county_df.csv', index=True, header=True)


In [10]:
#6. rates_by_state_df
#divide deaths by cases for state
rates_by_state_df = deaths_by_state_df.div(cases_by_state_df)
#rates_by_state_df=rates_by_state_df.fillna(0)
del rates_by_state_df['stateFIPS']
del rates_by_state_df['countyFIPS']
#del rates_by_state_df['population']
rates_by_state_df.to_csv('output_data/rates_by_state_df.csv', index=True, header=True)##per capita

In [11]:
#7. cases_by_county_per_capita
#divide by population
cases_by_county_per_capita = cases_by_county_df
list_of_columns = [column for column in cases_by_county_per_capita.columns if column not in ['countyFIPS', 'stateFIPS','population']]
for column in list_of_columns:
    cases_by_county_per_capita[column] = cases_by_county_per_capita[column]/cases_by_county_per_capita['population']
cases_by_county_per_capita.to_csv('output_data/cases_by_county_per_capita.csv', index=True, header=True)   

In [12]:
#8. deaths_by_county_per_capita
#divide by population
deaths_by_county_per_capita = deaths_by_county_df
list_of_columns = [column for column in deaths_by_county_per_capita.columns if column not in ['countyFIPS', 'stateFIPS','population']]
for column in list_of_columns:
    deaths_by_county_per_capita[column] = deaths_by_county_per_capita[column]/deaths_by_county_per_capita['population']
deaths_by_county_per_capita.to_csv('output_data/deaths_by_county_per_capita.csv', index=True, header=True)

In [13]:
#9. rates_by_county_per_capita
#divide by population
rates_by_county_per_capita = rates_by_county_df
list_of_columns = [column for column in rates_by_county_per_capita.columns if column not in ['countyFIPS', 'stateFIPS','population']]
for column in list_of_columns:
    rates_by_county_per_capita[column] = rates_by_county_per_capita[column]/rates_by_county_per_capita['population']
rates_by_county_per_capita.to_csv('output_data/rates_by_county_per_capita.csv', index=True, header=True)

In [14]:
#10. cases_by_state_per_capita
#divide by population
cases_by_state_per_capita = cases_by_state_df
list_of_columns = [column for column in cases_by_state_per_capita.columns if column not in ['countyFIPS', 'stateFIPS','population']]
for column in list_of_columns:
    cases_by_state_per_capita[column] = cases_by_state_per_capita[column]/cases_by_state_per_capita['population']
cases_by_state_per_capita.to_csv('output_data/cases_by_state_per_capita.csv', index=True, header=True)

In [15]:
#11. deaths_by_state_per_capita
#divide by population
deaths_by_state_per_capita = deaths_by_state_df
list_of_columns = [column for column in deaths_by_state_per_capita.columns if column not in ['countyFIPS', 'stateFIPS','population']]
for column in list_of_columns:
    deaths_by_state_per_capita[column] = deaths_by_state_per_capita[column]/deaths_by_state_per_capita['population']
deaths_by_state_per_capita.to_csv('output_data/deaths_by_state_per_capita.csv', index=True, header=True)

In [16]:
#12. rates_by_state_per_capita
#divide by population
rates_by_state_per_capita = rates_by_state_df
list_of_columns = [column for column in rates_by_state_per_capita.columns if column not in ['countyFIPS', 'stateFIPS','population']]
for column in list_of_columns:
    rates_by_state_per_capita[column] = rates_by_state_per_capita[column]/rates_by_state_per_capita['population']
rates_by_state_per_capita.to_csv('output_data/rates_by_state_per_capita.csv', index=True, header=True)